In [62]:
import sys; sys.path.append('../../wwa/'); from wwa import *

xn, xx, yn, yx = [19,40.5,-27.5,-15]

# Fitted trends in gridded data products

## Get gridded data

In [11]:
chirps = xr.concat([xr.open_dataset(fnm).groupby("time.season")["DJF"].sel(longitude = slice(xn,xx), latitude = slice(yn,yx)) for fnm in sorted(glob.glob("/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/chirps_05/*.nc"))], "time")
chirps = chirps.rename(precip = "pr", longitude = "lon", latitude = "lat")

chirps_qs = chirps.pr.resample(time = "QS-DEC").sum()
chirps_qs = chirps_qs.where(chirps_qs.sum("time") > 0)

chirps_qs = chirps_qs.groupby("time.season")["DJF"].sel(time = slice("1981",None))

chirps_qs.to_netcdf("sa-drought_chirps.nc")

In [53]:
mswep = xr.open_dataset('../../00_WWA_project_folder/ephemeral/southernafrica/mswep/precip_mswep_southernafrca_1979-2024.nc').pr.sel(time = slice("1979-03", None))
mswep_qs = mswep.resample(time = "QS-DEC").sum()
mswep_qs = mswep_qs.where(mswep_qs.sum("time") > 0).groupby("time.season")["DJF"]
mswep_qs.to_netcdf("sa-drought_mswep.nc")

In [64]:
wrap_lon(xr.open_dataset("../../00_WWA_project_folder/ephemeral/southernafrica/TAMSAT/origfiles/rfe1983_daily_SouthernAfrica.v3.1.nc")).rfe.sel(lon = slice(xn,xx), lat = slice(yn,yx)).groupby("time.serason")

<xarray.DataArray 'rfe' (time: 365, lat: 334, lon: 574)>
[69976340 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 19.01 19.05 19.09 19.12 ... 40.39 40.42 40.46 40.5
  * lat      (lat) float64 -27.49 -27.45 -27.41 -27.37 ... -15.08 -15.04 -15.0
  * time     (time) datetime64[ns] 1983-01-01 1983-01-02 ... 1983-12-31
Attributes:
    units:       mm
    long_name:   Rainfall Estimate
    short_name:  rfe

In [96]:
tamsat = xr.open_mfdataset("../../00_WWA_project_folder/ephemeral/southernafrica/TAMSAT/origfiles/rfe*_daily_SouthernAfrica.v3.1.nc").rfe.sel(lon = slice(xn,xx), lat = slice(yx,yn))
tamsat_qs = tamsat.resample(time = "QS-DEC").sum().groupby("time.season")["DJF"].sel(time = slice("1983",None)).rename("pr")
tamsat_qs.to_netcdf("sa-drought_tamsat.nc")